# Image Segmentation

## Learning Objectives

By the end of this notebook, you will be able to:
- Understand image segmentation concepts and applications
- Apply supervised and unsupervised thresholding
- Use the watershed algorithm for object separation
- Implement marker-based watershed segmentation
- Count and separate overlapping objects
- Build automated object counting systems

---

---

**⏱️ Estimated Time**: 90-120 minutes  
**📚 Level**: Intermediate to Advanced  
**📋 Prerequisites**: Completed notebooks 00-05

---

## Setup

Import required libraries:

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage

# Configure matplotlib
%matplotlib inline
plt.rcParams["figure.figsize"] = (14, 8)

print("Libraries imported successfully!")
print(f"OpenCV version: {cv.__version__}")

---

## Part 1: Introduction to Image Segmentation

### What is Image Segmentation?

**Image segmentation** is the process of partitioning an image into multiple segments (regions or objects). The goal is to simplify the image representation and make it more meaningful for analysis.

### Types of Segmentation:
- **Semantic Segmentation** - Classify each pixel into a category
- **Instance Segmentation** - Identify individual object instances
- **Thresholding-based** - Separate foreground from background
- **Region-based** - Group similar pixels together
- **Edge-based** - Find boundaries between regions

### Applications:
- **Medical Imaging**: Tumor detection, organ segmentation
- **Autonomous Driving**: Road, pedestrian, vehicle detection
- **Manufacturing**: Defect detection, quality control
- **Biology**: Cell counting, microscopy analysis
- **Satellite Imagery**: Land use classification

---

## Part 2: Thresholding for Segmentation

### Supervised Thresholding

In **supervised thresholding**, you manually select the threshold value based on your knowledge of the image.

In [ ]:
# Create a sample image with objects
supervised_img = np.ones((300, 400), dtype=np.uint8) * 50  # Dark background

# Add objects with different intensities
cv.circle(supervised_img, (100, 100), 40, 150, -1)
cv.circle(supervised_img, (200, 100), 40, 180, -1)
cv.circle(supervised_img, (300, 100), 40, 200, -1)
cv.rectangle(supervised_img, (50, 180), (150, 250), 160, -1)
cv.rectangle(supervised_img, (200, 180), (350, 250), 190, -1)

# Try different manual thresholds
_, thresh_100 = cv.threshold(supervised_img, 100, 255, cv.THRESH_BINARY)
_, thresh_140 = cv.threshold(supervised_img, 140, 255, cv.THRESH_BINARY)
_, thresh_170 = cv.threshold(supervised_img, 170, 255, cv.THRESH_BINARY)

# Display
plt.figure(figsize=(18, 10))

plt.subplot(2, 2, 1)
plt.imshow(supervised_img, cmap="gray")
plt.title("Original Image", fontsize=14)
plt.axis("off")

plt.subplot(2, 2, 2)
plt.imshow(thresh_100, cmap="gray")
plt.title("Threshold = 100\n(All objects captured)", fontsize=12)
plt.axis("off")

plt.subplot(2, 2, 3)
plt.imshow(thresh_140, cmap="gray")
plt.title("Threshold = 140\n(Some objects lost)", fontsize=12)
plt.axis("off")

plt.subplot(2, 2, 4)
plt.imshow(thresh_170, cmap="gray")
plt.title("Threshold = 170\n(Only brightest objects)", fontsize=12)
plt.axis("off")

plt.tight_layout()
plt.show()

print("Supervised thresholding requires manual threshold selection")

### 💡 Parameter Tuning Tips for Watershed Segmentation

**Distance Transform Threshold** (0.0-1.0):
- Lower (0.3-0.5): More markers, more segments (over-segmentation)
- Higher (0.6-0.8): Fewer markers, merged segments (under-segmentation)
- Typical: 0.5-0.7 for most applications

**Morphological Operations (before watershed)**:
- More iterations: Cleaner background, but may merge close objects
- Fewer iterations: Preserves object boundaries, but noisier

**Connectivity** (for connected components):
- 4-connectivity: Conservative, may split diagonal connections
- 8-connectivity: Includes diagonals, more connected regions

**When to adjust**:
- Over-segmentation (too many regions): Increase distance threshold, more erosion iterations
- Under-segmentation (merged objects): Decrease distance threshold, less erosion

### 💡 Parameter Tuning Tips for Watershed Segmentation

**Distance Transform Threshold** (0.0-1.0):
- Lower (0.3-0.5): More markers, more segments (over-segmentation)
- Higher (0.6-0.8): Fewer markers, merged segments (under-segmentation)
- Typical: 0.5-0.7 for most applications

**Morphological Operations (before watershed)**:
- More iterations: Cleaner background, but may merge close objects
- Fewer iterations: Preserves object boundaries, but noisier

**Connectivity** (for connected components):
- 4-connectivity: Conservative, may split diagonal connections
- 8-connectivity: Includes diagonals, more connected regions

**When to adjust**:
- Over-segmentation (too many regions): Increase distance threshold, more erosion iterations
- Under-segmentation (merged objects): Decrease distance threshold, less erosion

---

## Part 3: Watershed Algorithm - Theory

### What is the Watershed Algorithm?

The **watershed algorithm** treats the image as a topographic surface:
- **Dark pixels** = Valleys (low elevation)
- **Bright pixels** = Peaks (high elevation)
- **Watershed lines** = Boundaries between regions

### How it Works:
1. Start filling water from the valleys (local minima)
2. Water from different valleys will meet at watershed lines
3. These lines represent object boundaries

### Problem: Over-segmentation
- Watershed can create too many regions due to noise
- **Solution**: Use **markers** to guide segmentation

### Marker-Based Watershed:
1. **Sure foreground** - Definitely part of objects
2. **Sure background** - Definitely background
3. **Unknown region** - Boundary area to be determined

---

## Part 4: Basic Watershed Example

In [ ]:
# Create image with overlapping circles
coins_img = np.zeros((300, 400), dtype=np.uint8)

# Add overlapping circles (simulating coins)
cv.circle(coins_img, (100, 150), 50, 255, -1)
cv.circle(coins_img, (160, 150), 50, 255, -1)
cv.circle(coins_img, (130, 210), 50, 255, -1)
cv.circle(coins_img, (300, 150), 45, 255, -1)

# Visualize the "topographic" view
plt.figure(figsize=(18, 5))

plt.subplot(1, 3, 1)
plt.imshow(coins_img, cmap="gray")
plt.title("Overlapping Circles (Coins)", fontsize=14)
plt.axis("off")

plt.subplot(1, 3, 2)
plt.imshow(coins_img, cmap="terrain")
plt.title("Topographic View (Watershed Analogy)", fontsize=14)
plt.colorbar(label="Elevation")
plt.axis("off")

# Count connected components (before watershed)
num_labels, labels = cv.connectedComponents(coins_img)
plt.subplot(1, 3, 3)
plt.imshow(labels, cmap="nipy_spectral")
plt.title(f"Simple Thresholding: {num_labels-1} objects\n(Should be 4!)", fontsize=12)
plt.axis("off")

plt.tight_layout()
plt.show()

print(f"Without watershed: Found {num_labels-1} objects (incorrect!)")
print("The touching circles are counted as one object")

---

## Part 5: Marker-Based Watershed Implementation

### Step-by-Step Process:

In [ ]:
# Step 1: Noise removal using morphology
kernel = np.ones((3, 3), np.uint8)
opening = cv.morphologyEx(coins_img, cv.MORPH_OPEN, kernel, iterations=2)

# Step 2: Sure background area (dilate)
sure_bg = cv.dilate(opening, kernel, iterations=3)

# Step 3: Sure foreground area using distance transform
dist_transform = cv.distanceTransform(opening, cv.DIST_L2, 5)
_, sure_fg = cv.threshold(dist_transform, 0.5 * dist_transform.max(), 255, 0)
sure_fg = np.uint8(sure_fg)

# Step 4: Unknown region (background - foreground)
unknown = cv.subtract(sure_bg, sure_fg)

# Visualize the regions
plt.figure(figsize=(20, 10))

plt.subplot(2, 3, 1)
plt.imshow(coins_img, cmap="gray")
plt.title("1. Original Image", fontsize=12)
plt.axis("off")

plt.subplot(2, 3, 2)
plt.imshow(opening, cmap="gray")
plt.title("2. After Opening (noise removed)", fontsize=12)
plt.axis("off")

plt.subplot(2, 3, 3)
plt.imshow(sure_bg, cmap="gray")
plt.title("3. Sure Background (dilated)", fontsize=12)
plt.axis("off")

plt.subplot(2, 3, 4)
plt.imshow(dist_transform, cmap="hot")
plt.title("4. Distance Transform", fontsize=12)
plt.colorbar()
plt.axis("off")

plt.subplot(2, 3, 5)
plt.imshow(sure_fg, cmap="gray")
plt.title("5. Sure Foreground (peaks)", fontsize=12)
plt.axis("off")

plt.subplot(2, 3, 6)
plt.imshow(unknown, cmap="gray")
plt.title("6. Unknown Region (to segment)", fontsize=12)
plt.axis("off")

plt.tight_layout()
plt.show()

print("Prepared regions for watershed segmentation")

In [ ]:
# Step 5: Label markers
_, markers = cv.connectedComponents(sure_fg)

# Step 6: Add 1 to all labels so background is not 0, but 1
markers = markers + 1

# Step 7: Mark unknown region as 0
markers[unknown == 255] = 0

# Visualize markers
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.imshow(sure_fg, cmap="gray")
plt.title("Sure Foreground", fontsize=14)
plt.axis("off")

plt.subplot(1, 3, 2)
plt.imshow(markers, cmap="nipy_spectral")
plt.title(f"Markers (Before Watershed)\n{markers.max()} regions marked", fontsize=12)
plt.colorbar()
plt.axis("off")

# Step 8: Apply watershed (need color image)
coins_color = cv.cvtColor(coins_img, cv.COLOR_GRAY2BGR)
markers_watershed = cv.watershed(coins_color, markers)

plt.subplot(1, 3, 3)
plt.imshow(markers_watershed, cmap="nipy_spectral")
plt.title(f"After Watershed\nBoundaries marked as -1", fontsize=12)
plt.colorbar()
plt.axis("off")

plt.tight_layout()
plt.show()

print(f"Watershed found {markers.max()-1} objects")
print(f"Boundary pixels marked as -1")

In [ ]:
# Step 9: Mark boundaries in red on original image
result = coins_color.copy()
result[markers_watershed == -1] = [0, 0, 255]  # Red boundaries

# Draw bounding boxes around each object
result_boxes = coins_color.copy()
for marker_id in range(2, markers.max() + 1):
    mask = np.uint8(markers_watershed == marker_id)
    contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    if contours:
        x, y, w, h = cv.boundingRect(contours[0])
        cv.rectangle(result_boxes, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv.putText(
            result_boxes,
            str(marker_id - 1),
            (x + 5, y + 20),
            cv.FONT_HERSHEY_SIMPLEX,
            0.6,
            (255, 255, 0),
            2,
        )

# Display final results
plt.figure(figsize=(18, 5))

plt.subplot(1, 3, 1)
plt.imshow(cv.cvtColor(coins_color, cv.COLOR_BGR2RGB))
plt.title("Original Overlapping Circles", fontsize=14)
plt.axis("off")

plt.subplot(1, 3, 2)
plt.imshow(cv.cvtColor(result, cv.COLOR_BGR2RGB))
plt.title("Watershed Boundaries (Red)", fontsize=14)
plt.axis("off")

plt.subplot(1, 3, 3)
plt.imshow(cv.cvtColor(result_boxes, cv.COLOR_BGR2RGB))
plt.title(f"Detected Objects: {markers.max()-1}", fontsize=14)
plt.axis("off")

plt.tight_layout()
plt.show()

print(f"✓ Successfully separated {markers.max()-1} overlapping objects!")

---

## Part 6: Coin Counting Application

Let's build a more realistic coin counting example:

In [ ]:
# Create more realistic coin image
coin_scene = np.zeros((400, 500), dtype=np.uint8)

# Add multiple overlapping coins with slight variations
np.random.seed(42)
coin_positions = [
    (80, 100, 45),
    (150, 110, 48),
    (90, 180, 46),
    (180, 190, 44),
    (260, 120, 50),
    (340, 130, 47),
    (290, 210, 45),
    (380, 220, 46),
    (430, 140, 48),
    (150, 290, 44),
    (240, 310, 49),
    (350, 320, 45),
]

for x, y, r in coin_positions:
    intensity = np.random.randint(200, 240)
    cv.circle(coin_scene, (x, y), r, intensity, -1)

# Add some noise
noise = np.random.normal(0, 10, coin_scene.shape).astype(np.int16)
coin_scene = np.clip(coin_scene.astype(np.int16) + noise, 0, 255).astype(np.uint8)

# Apply watershed
# 1. Threshold
_, binary = cv.threshold(coin_scene, 100, 255, cv.THRESH_BINARY)

# 2. Noise removal
kernel = np.ones((3, 3), np.uint8)
opening = cv.morphologyEx(binary, cv.MORPH_OPEN, kernel, iterations=2)

# 3. Sure background
sure_bg = cv.dilate(opening, kernel, iterations=3)

# 4. Distance transform for sure foreground
dist_transform = cv.distanceTransform(opening, cv.DIST_L2, 5)
_, sure_fg = cv.threshold(dist_transform, 0.4 * dist_transform.max(), 255, 0)
sure_fg = np.uint8(sure_fg)

# 5. Unknown region
unknown = cv.subtract(sure_bg, sure_fg)

# 6. Marker labeling
_, markers = cv.connectedComponents(sure_fg)
markers = markers + 1
markers[unknown == 255] = 0

# 7. Watershed
coin_color = cv.cvtColor(coin_scene, cv.COLOR_GRAY2BGR)
markers = cv.watershed(coin_color, markers)

# 8. Count and mark coins
coin_result = coin_color.copy()
coin_result[markers == -1] = [0, 0, 255]

num_coins = markers.max() - 1

# Draw circles around each coin
for marker_id in range(2, markers.max() + 1):
    mask = np.uint8(markers == marker_id)
    contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    if contours:
        (x, y), radius = cv.minEnclosingCircle(contours[0])
        cv.circle(coin_result, (int(x), int(y)), int(radius), (0, 255, 0), 2)
        cv.putText(
            coin_result,
            str(marker_id - 1),
            (int(x) - 10, int(y) + 5),
            cv.FONT_HERSHEY_SIMPLEX,
            0.6,
            (255, 255, 0),
            2,
        )

# Display
plt.figure(figsize=(18, 5))

plt.subplot(1, 3, 1)
plt.imshow(coin_scene, cmap="gray")
plt.title("Coin Image (with overlaps)", fontsize=14)
plt.axis("off")

plt.subplot(1, 3, 2)
plt.imshow(markers, cmap="nipy_spectral")
plt.title(f"Watershed Segmentation\n{num_coins} regions found", fontsize=12)
plt.axis("off")

plt.subplot(1, 3, 3)
plt.imshow(cv.cvtColor(coin_result, cv.COLOR_BGR2RGB))
plt.title(f"Coin Counter Result\nTotal Coins: {num_coins}", fontsize=14, fontweight="bold")
plt.axis("off")

plt.tight_layout()
plt.show()

print(f"\n{'='*50}")
print(f"COIN COUNTING RESULT: {num_coins} coins detected")
print(f"{'='*50}")

---

## Part 7: Cell Counting Application

Watershed is particularly useful in microscopy for counting cells:

In [ ]:
# Simulate microscopy image with cells
cell_img = np.zeros((400, 500), dtype=np.uint8)

# Add cell-like circles
np.random.seed(123)
cell_positions = []
for _ in range(25):
    x = np.random.randint(40, 460)
    y = np.random.randint(40, 360)
    r = np.random.randint(18, 28)
    cell_positions.append((x, y, r))

for x, y, r in cell_positions:
    intensity = np.random.randint(180, 230)
    cv.circle(cell_img, (x, y), r, intensity, -1)
    # Add nucleus (brighter center)
    cv.circle(
        cell_img,
        (x + np.random.randint(-3, 3), y + np.random.randint(-3, 3)),
        int(r * 0.4),
        250,
        -1,
    )

# Add background texture
noise = np.random.normal(30, 15, cell_img.shape).astype(np.int16)
cell_img = np.clip(cell_img.astype(np.int16) + noise, 0, 255).astype(np.uint8)

# Apply Gaussian blur to smooth
cell_img = cv.GaussianBlur(cell_img, (5, 5), 0)

# Watershed segmentation
# 1. Otsu thresholding
_, binary = cv.threshold(cell_img, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)

# 2. Noise removal
kernel = np.ones((3, 3), np.uint8)
opening = cv.morphologyEx(binary, cv.MORPH_OPEN, kernel, iterations=2)

# 3. Sure background
sure_bg = cv.dilate(opening, kernel, iterations=3)

# 4. Sure foreground (distance transform)
dist_transform = cv.distanceTransform(opening, cv.DIST_L2, 5)
_, sure_fg = cv.threshold(dist_transform, 0.5 * dist_transform.max(), 255, 0)
sure_fg = np.uint8(sure_fg)

# 5. Unknown region
unknown = cv.subtract(sure_bg, sure_fg)

# 6. Markers
_, markers = cv.connectedComponents(sure_fg)
markers = markers + 1
markers[unknown == 255] = 0

# 7. Watershed
cell_color = cv.cvtColor(cell_img, cv.COLOR_GRAY2BGR)
markers = cv.watershed(cell_color, markers)

# 8. Analyze results
cell_result = cell_color.copy()
cell_result[markers == -1] = [0, 0, 255]

num_cells = markers.max() - 1

# Calculate cell sizes
cell_sizes = []
for marker_id in range(2, markers.max() + 1):
    mask = np.uint8(markers == marker_id)
    area = np.sum(mask)
    cell_sizes.append(area)

    # Draw contour
    contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    if contours:
        cv.drawContours(cell_result, contours, -1, (0, 255, 0), 2)

# Display
plt.figure(figsize=(20, 10))

plt.subplot(2, 3, 1)
plt.imshow(cell_img, cmap="gray")
plt.title("Microscopy Image (Simulated)", fontsize=14)
plt.axis("off")

plt.subplot(2, 3, 2)
plt.imshow(binary, cmap="gray")
plt.title("After Otsu Thresholding", fontsize=12)
plt.axis("off")

plt.subplot(2, 3, 3)
plt.imshow(dist_transform, cmap="hot")
plt.title("Distance Transform", fontsize=12)
plt.colorbar()
plt.axis("off")

plt.subplot(2, 3, 4)
plt.imshow(sure_fg, cmap="gray")
plt.title("Sure Foreground (Seeds)", fontsize=12)
plt.axis("off")

plt.subplot(2, 3, 5)
plt.imshow(markers, cmap="nipy_spectral")
plt.title(f"Watershed Result\n{num_cells} cells segmented", fontsize=12)
plt.axis("off")

plt.subplot(2, 3, 6)
plt.imshow(cv.cvtColor(cell_result, cv.COLOR_BGR2RGB))
plt.title(f"Cell Detection\nTotal: {num_cells} cells", fontsize=14, fontweight="bold")
plt.axis("off")

plt.tight_layout()
plt.show()

# Statistics
print(f"\n{'='*60}")
print(f"CELL ANALYSIS REPORT")
print(f"{'='*60}")
print(f"Total cells detected: {num_cells}")
print(f"Average cell size: {np.mean(cell_sizes):.1f} pixels")
print(f"Smallest cell: {np.min(cell_sizes)} pixels")
print(f"Largest cell: {np.max(cell_sizes)} pixels")
print(f"Standard deviation: {np.std(cell_sizes):.1f} pixels")
print(f"{'='*60}")

---

## Part 8: Comparison - Simple Threshold vs Watershed

In [ ]:
# Create challenging image with many overlapping objects
challenge_img = np.zeros((300, 400), dtype=np.uint8)

# Add many overlapping circles
circles = [
    (80, 80, 40),
    (130, 80, 38),
    (180, 85, 42),
    (100, 140, 39),
    (155, 145, 41),
    (95, 200, 40),
    (160, 205, 38),
    (250, 100, 45),
    (300, 110, 43),
    (270, 180, 40),
    (330, 190, 42),
]

for x, y, r in circles:
    cv.circle(challenge_img, (x, y), r, 220, -1)

# Method 1: Simple connected components
_, binary_simple = cv.threshold(challenge_img, 100, 255, cv.THRESH_BINARY)
num_simple, labels_simple = cv.connectedComponents(binary_simple)

# Method 2: Watershed
kernel = np.ones((3, 3), np.uint8)
opening = cv.morphologyEx(binary_simple, cv.MORPH_OPEN, kernel, iterations=2)
sure_bg = cv.dilate(opening, kernel, iterations=3)
dist_transform = cv.distanceTransform(opening, cv.DIST_L2, 5)
_, sure_fg = cv.threshold(dist_transform, 0.5 * dist_transform.max(), 255, 0)
sure_fg = np.uint8(sure_fg)
unknown = cv.subtract(sure_bg, sure_fg)
_, markers = cv.connectedComponents(sure_fg)
markers = markers + 1
markers[unknown == 255] = 0
challenge_color = cv.cvtColor(challenge_img, cv.COLOR_GRAY2BGR)
markers = cv.watershed(challenge_color, markers)
num_watershed = markers.max() - 1

# Visualize comparison
result_simple = challenge_color.copy()
result_watershed = challenge_color.copy()
result_watershed[markers == -1] = [0, 0, 255]

plt.figure(figsize=(18, 10))

plt.subplot(2, 3, 1)
plt.imshow(challenge_img, cmap="gray")
plt.title("Original: 11 Overlapping Objects", fontsize=14, fontweight="bold")
plt.axis("off")

plt.subplot(2, 3, 2)
plt.imshow(labels_simple, cmap="nipy_spectral")
plt.title(f"Simple Threshold\nDetected: {num_simple-1} objects ❌", fontsize=12, color="red")
plt.axis("off")

plt.subplot(2, 3, 3)
plt.imshow(cv.cvtColor(result_simple, cv.COLOR_BGR2RGB))
plt.title("Simple Threshold Result\n(Fails with overlaps)", fontsize=12)
plt.axis("off")

plt.subplot(2, 3, 4)
plt.imshow(dist_transform, cmap="hot")
plt.title("Distance Transform\n(Shows object centers)", fontsize=12)
plt.colorbar()
plt.axis("off")

plt.subplot(2, 3, 5)
plt.imshow(markers, cmap="nipy_spectral")
plt.title(
    f"Watershed Segmentation\nDetected: {num_watershed} objects ✓", fontsize=12, color="green"
)
plt.axis("off")

plt.subplot(2, 3, 6)
plt.imshow(cv.cvtColor(result_watershed, cv.COLOR_BGR2RGB))
plt.title("Watershed Result\n(Separates overlaps!)", fontsize=12)
plt.axis("off")

plt.tight_layout()
plt.show()

print(f"\n{'='*60}")
print(f"COMPARISON RESULTS")
print(f"{'='*60}")
print(f"Ground truth: 11 objects")
print(f"Simple threshold: {num_simple-1} objects (INCORRECT)")
print(f"Watershed: {num_watershed} objects (CORRECT!)")
print(f"\nWatershed successfully separates overlapping objects!")
print(f"{'='*60}")

---

## Part 9: Practical Exercises

### Exercise 1: Adjust Watershed Parameters

In [ ]:
# TODO: Take the coin counting example above
# Experiment with different parameters:
# - Distance transform threshold (0.3, 0.4, 0.5, 0.6, 0.7)
# - Morphological opening iterations
# - Dilation iterations for sure background
# See how each parameter affects segmentation quality

print("Try different parameter values and observe the results!")
print("Hint: Lower distance threshold = more aggressive separation")
print("Hint: Higher distance threshold = more conservative separation")

### Exercise 2: Build an Automated Object Counter

In [ ]:
# TODO: Create a function that takes an image and returns:
# - Number of objects
# - Average object size
# - List of object areas
# - Visualized result with labels


def count_objects(image, dist_threshold=0.5):
    """
    Count objects in an image using watershed segmentation.

    Parameters:
    - image: Input grayscale image
    - dist_threshold: Distance transform threshold (0-1)

    Returns:
    - num_objects: Number of detected objects
    - areas: List of object areas
    - result_image: Segmented result image
    """
    # Your implementation here!
    pass


print("Implement the count_objects function!")

### Exercise 3: Handle Different Object Types

In [ ]:
# TODO: Create an image with:
# - Different sized objects (small, medium, large)
# - Different shapes (circles, ellipses, rectangles)
# - Varying brightness levels
# Apply watershed and see how well it handles diversity

print("Test watershed robustness with diverse objects!")

---

## Summary

Congratulations! You've completed Image Segmentation. You now know:

✓ Image segmentation concepts and applications  
✓ Supervised vs unsupervised thresholding  
✓ Otsu's automatic thresholding  
✓ Watershed algorithm theory and implementation  
✓ Marker-based watershed segmentation  
✓ Distance transform for marker creation  
✓ Separating overlapping objects  
✓ Object counting and size analysis  

### Key Takeaways

1. **Watershed treats image as topography** - valleys and peaks
2. **Markers guide segmentation** - prevents over-segmentation
3. **Distance transform finds object centers** - creates good markers
4. **Morphology is essential** - opening removes noise, dilation finds background
5. **Watershed separates touching objects** - where simple threshold fails
6. **Parameters matter** - distance threshold controls separation aggressiveness
7. **Works best on blob-like objects** - circles, cells, coins

---

## Watershed Algorithm Steps Summary

1. **Preprocessing** - Threshold, denoise with opening
2. **Sure background** - Dilate to get definite background
3. **Distance transform** - Find distance to nearest background pixel
4. **Sure foreground** - Threshold distance transform to get object centers
5. **Unknown region** - Subtract foreground from background
6. **Create markers** - Label connected components
7. **Apply watershed** - Find boundaries between markers
8. **Extract results** - Count objects, analyze sizes, draw boundaries

---

## What's Next?

In the next notebook (**07_feature_detection.ipynb**), you'll learn:
- Corner detection (Harris, Shi-Tomasi)
- Scale-invariant features (SIFT)
- Fast feature detection (FAST, ORB)
- Feature descriptors and keypoints

---

## Real-World Applications

- **Medical Imaging**: Cell counting, tumor segmentation, tissue analysis
- **Manufacturing**: Part counting, defect detection, quality inspection
- **Agriculture**: Fruit counting, crop analysis, yield estimation
- **Biology**: Microscopy analysis, colony counting, organism tracking
- **Materials Science**: Particle analysis, grain size measurement

---

**Happy Coding!** 🔬📊